# Liste mit möglichen Parsern für die Doku

## HTML.parser Paket

https://www.educative.io/edpresso/what-is-the-html-parser-in-python

Der folgende Code ist aus dem Link übernommen und nur leicht an unsere Problemstellung angepasst

In [ ]:
from html.parser import HTMLParser

In [ ]:
Hier werden Funktionen zum Auslesen der HTML-Elemente erstellt

In [ ]:
class Parser(HTMLParser):
  # method to append the start tag to the list start_tags.
  def handle_starttag(self, tag, attrs):
    global start_tags
    start_tags.append(tag)
    # method to append the end tag to the list end_tags.
  def handle_endtag(self, tag):
    global end_tags
    end_tags.append(tag)
  # method to append the data between the tags to the list all_data.
  def handle_data(self, data):
    global all_data
    all_data.append(data)
  # method to append the comment to the list comments.
  def handle_comment(self, data):
    global comments
    comments.append(data)

In [ ]:
Nun kann eine HTML-Datei ausgelesen werden.
Die möglichen Ergebnisse werden in Listen gespeichert

In [ ]:
start_tags = []
end_tags = []
all_data = []
comments = []
# Creating an instance of our class.
parser = Parser()
# Poviding the input.
HTMLFile = open("path/to/file.html", "r", encoding = "utf-8")
parser.feed(HTMLFile.read())
#print("start tags:", start_tags)
#print("end tags:", end_tags)
#print("data:", all_data)
#print("comments", comments)

Da die Liste "data" alle Zeichen aus dem Body der Seite enthält, sollten zunächst die Seitenumbrüche entfernt werden

In [ ]:
data =data.replace("\n"," ")

## Beautiful Soup

https://www.twilio.com/blog/web-scraping-and-parsing-html-in-python-with-beautiful-soup

 Im folgenden Code wurde versucht eine URL zu den Seiten zu verwenden
 
 Problem:
    
    * meisten Parser lesen eine HTML Seite aus dem Browser aus
    * Unsere lokale Webseite wird also nicht abgedeckt

In [ ]:
import requests
from bs4 import BeautifulSoup


vgm_url = 'path/to/file.html'
html_text = requests.get(vgm_url).text
soup = BeautifulSoup(html_text, 'html.parser')


### Beautiful soup local
    
    https://www.geeksforgeeks.org/how-to-parse-local-html-file-in-python/
    
    - über das BeatifulSoup Paket ist es möglich HTML-Elemente direkt auszulesen, so können Überschriften, Links und Co. getrennt betrachtet werden

In [ ]:
# Importing BeautifulSoup class from the bs4 module
from bs4 import BeautifulSoup

In [ ]:
# Opening the html file
HTMLFile = open("path/to/file.html", "r", encoding = "utf-8")

In [ ]:
# Reading the file
index = HTMLFile.read()

*Wichtig*: Hier wird das eigentliche BeatifulSoup Element erstellt, auf dem später gearbeitet werden kann

In [ ]:
# Creating a BeautifulSoup object and specifying the parser
S = BeautifulSoup(index,'html.parser')

Hier kann der aus der HTML-Datei gelesene Code schon präsentiert werden (hilft beim Überprüfen, ob Daten korrekt übernommen)

In [ ]:
# Using the prettify method to modify the code
print(S.body.prettify())

Es ist möglich verschiedene Elemente des HTML gezielt anzusteuern, hier einmal den Header und eine H1 Überschrift

In [ ]:
print(S.head)

In [ ]:
print(S.h1.get_text())

Hier werden alle Links die im HTML-File zu finden sind in einer Liste gespeichert

In [ ]:
all_link = []
for link in S.find_all('a'):
    all_link.append(link.get('href'))
    print(link.get('href'))

Über *get_text()* können alle Textelemente aus dem Text gezogen werden (inklusive Tabellen-Inhalte, Texte zu Links, etc.)

In [ ]:
print(S.get_text())

 Die meisten sinnvollen Text-Inhalte werden in p-Blöcken zusammengefasst, diese kann man extrahieren

In [ ]:
all_p = []
for text in S.find_all('p'):
    all_p.append(text.get_text())
    print(text.get_text())
print(all_p)

Eventuell wichtige Meta-Informationen sind die Unterkapitel die sich in einem HTML_File befinden

In [ ]:
all_unter = []
for unterkap in S.find_all('h2'):
    all_unter.append(unterkap.get_text())
    print(unterkap.get_text())

Da kein alternativ Text zu den Bildern existiert, können zumindest die Datei-Pfade erfasst werden

In [ ]:
all_bild=[]
for bild in S.find_all('img'):
    all_bild.append(bild.get('src'))

print(all_bild)

Eine weitere Möglichkeit den Body-Text für die HTML-Datei zu erhalten

In [ ]:
data_text = S.get_text()

Da der gesamte Text-Inhalt übernommen wird, entstehen viele unnötige Zeilenumbrüche, die es rauszunehmen gilt:

In [ ]:
import re

data_text =re.sub(r'\n\s*\n', r'\n\n', S.get_text().strip(), flags=re.M)
#print(data_text)

### Mögliches Speichern unserer Ausgelesenen Daten

- speichern der Daten als CSV-Datei

#### Aufbau:

|Titel|Unterkapitel|Text-Körper|Links|Bilder|
|-----|------------|-----------|-----|------|

- dabei ist der Text-Körper zunächst aus den p-Blöcken des HTML-Files aufgebaut

In [ ]:
import csv

header = ['title','unterkapitel', 'body', 'links','bilder']
data = [S.h1.get_text(),all_unter,all_p,all_link, all_bild]

with open ('test_file1.csv','w',encoding='utf-8') as f:
    writer = csv.writer(f)
    #write header
    writer.writerow(header)
    #write the data
    writer.writerow(data)

##### Weitere Möglichkeit:

###### Aufbau:

|Titel|Unterkapitel|Text-Körper|Links|Bilder|
|-----|------------|-----------|-----|------|

- hier werden als Text-Body alle Text-Elemente der HTML-Seite genommen

In [ ]:
header = ['title','unterkapitel', 'body', 'links','bilder']
data = [S.h1.get_text(),all_unter,data_text,all_link, all_bild]

with open ('test_file2.csv','w',encoding='utf-8') as f:
    writer = csv.writer(f)
    #write header
    writer.writerow(header)
    #write the data
    writer.writerow(data)


### Ein bisschen befüllen der CSV

 - weitere Möglichkeit für eine Speicherung des HTML-Inhaltes

In [ ]:
HTMLFile2 = open("path/to/file.html", "r", encoding = "utf-8")
index2 = HTMLFile2.read()
# Creating a BeautifulSoup object and specifying the parser
soup = BeautifulSoup(index2,'html.parser')
# Using the select-one method to find the second element from the li tag
Tag2 = soup.select_one('li:nth-of-type(2)')
# Using the decompose method
Tag2.decompose()

Bereinigen der Text-Elemente:

In [ ]:
data_text2 =re.sub(r'\n\s*\n', r'\n\n', soup.get_text().strip(), flags=re.M)
print(data_text2)

Versuch die Tabellen-Elemente aus dem HTML-File zu extrahieren

In [ ]:
soup.table

In [ ]:
tabelle = []
table_body = soup.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    tabelle.append([ele for ele in cols if ele]) # Get rid of empty values


#### Fazit:

- Tabellen Elemente die nur aus Sonderzeichen oder kurzen Beschreibungen bestehen sind schwer zu extrahieren
- BeautifulSoup liest die Elemente nacheinander aus, wodurch der Zusammenhang der Elemente verloren geht
- Daher die Frage: Wie wichtig sind solche Tabellen, wie im Tools-Bereich der Dokumentation für unser Anliegen?